#Importante: en todos los algoritmos revisar las rutas de las carpeta y adecuar losa algoritmo para uso en diferentes escenarios.

In [1]:
#@title **Archivos necesarios para el preprocesamiento**
import gdown
import os
def descarga(diccionario, directorio):
    # Extrae el nombre del archivo y el enlace del diccionario
    nombre_archivo = diccionario['nombre']
    enlace = diccionario['enlace']

    # Extrae el hash del enlace
    file_id = enlace.split('/')[-2]

    # Genera la URL de descarga directa
    url_descarga = f'https://drive.google.com/uc?id={file_id}'

    # Directorio de destino para la descarga
    directorio_destino = directorio

    # Descarga el archivo en el directorio de destino con el nombre original
    archivo_descargado = os.path.join(directorio_destino, nombre_archivo)
    gdown.download(url_descarga, output=archivo_descargado, quiet=False)

    print(f'Archivo descargado desde {enlace} como {nombre_archivo}')

# Lista de enlaces y nombres de archivo en forma de diccionarios
enlaces = [
    {'nombre': 'requirements.txt', 'enlace': 'https://drive.google.com/file/d/1sFLw-0OUD76Xnd1MPzr3Xau6LNGu3Qjw/view?usp=share_link'},
    {'nombre': 'helpers.py', 'enlace': 'https://drive.google.com/file/d/1puT1QhGpWDB1C8EXY7QwGPWuqdOAPrfG/view?usp=share_link'},
    {'nombre': 'PD25-subcortical-1mm.nii.gz', 'enlace': 'https://drive.google.com/file/d/1jZdFkirBmEm5zf3wtSw2WKvYtvzrT_hL/view?usp=drive_link'},
]
#Cambiar el directorio a donde se quiere que se descargen los archivos:
directorio = '/home/franklin_pupils/yefersonValencia/MRIwithCyclegan'

for diccionario in enlaces:
    descarga(diccionario, directorio)

Downloading...
From: https://drive.google.com/uc?id=1sFLw-0OUD76Xnd1MPzr3Xau6LNGu3Qjw
To: /home/franklin_pupils/yefersonValencia/MRIwithCyclegan/requirements.txt
100%|██████████| 90.0/90.0 [00:00<00:00, 384kB/s]


Archivo descargado desde https://drive.google.com/file/d/1sFLw-0OUD76Xnd1MPzr3Xau6LNGu3Qjw/view?usp=share_link como requirements.txt


Downloading...
From (uriginal): https://drive.google.com/uc?id=1puT1QhGpWDB1C8EXY7QwGPWuqdOAPrfG
From (redirected): https://drive.google.com/uc?id=1puT1QhGpWDB1C8EXY7QwGPWuqdOAPrfG&confirm=t&uuid=09c8e4b5-fb2a-4c4f-ba56-9f990c1b6e7a
To: /home/franklin_pupils/yefersonValencia/MRIwithCyclegan/helpers.py
100%|██████████| 4.26k/4.26k [00:00<00:00, 721kB/s]


Archivo descargado desde https://drive.google.com/file/d/1puT1QhGpWDB1C8EXY7QwGPWuqdOAPrfG/view?usp=share_link como helpers.py


Downloading...
From: https://drive.google.com/uc?id=1jZdFkirBmEm5zf3wtSw2WKvYtvzrT_hL
To: /home/franklin_pupils/yefersonValencia/MRIwithCyclegan/PD25-subcortical-1mm.nii.gz
100%|██████████| 45.8k/45.8k [00:00<00:00, 139MB/s]

Archivo descargado desde https://drive.google.com/file/d/1jZdFkirBmEm5zf3wtSw2WKvYtvzrT_hL/view?usp=drive_link como PD25-subcortical-1mm.nii.gz


In [2]:
#@title **Instalar librerias necesarias para el preprocesamiento**

%pip install nilearn
%pip install nibabel
%pip install -r /home/franklin_pupils/franklin_pupils/yeferson/MRIwithCyclegan/requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
#@title **Cargar librerias necesarias para el preprocesamiento**
import os
import sys
import sys
sys.path.append('/home/franklin_pupils/franklin_pupils/yeferson/MRIwithCyclegan/')
from helpers import *

from nilearn import datasets
from nilearn import image as nimg
from nilearn import plotting as nplot
import nibabel as nib
from nilearn.maskers import nifti_labels_masker
import matplotlib.pyplot as plt
import nibabel as nib
import pandas as pd
import seaborn as sns
import numpy as np
import ants
import SimpleITK as sitk
from antspynet.utilities import brain_extraction
print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

2023-12-15 19:47:00.227653: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-15 19:47:00.303571: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AntsPy version = 0.4.2
SimpleITK version = 2.3.1


In [ ]:
#@title **Visualizacion de archivos**

#Uso: Se proporciona el la ruta del archivo normalizado y la ruta del archivo con la mascara aplicada

# Ver un archivo y su mascara

#Obtener la ruta del archivo de anat .nii.gz
anat_file_origin = ants.image_read(
    "/content/drive/MyDrive/neurocon/data/raw/parkinson/sub-patient032030/anat/sub-patient032030_T1w.nii.gz",
    reorient="IAL")
anat_file = ants.image_read(
    "/content/drive/MyDrive/neurocon/data/preprocessed/parkinson/sub-patient032030/anat/sub-patient032030_T1w_preprocessed.nii.gz",
    reorient="IAL")
mask_file_aplied = ants.image_read(
    "/content/drive/MyDrive/neurocon/data/preprocessed/parkinson/sub-patient032030/anat/sub-patient032030_T1w_preprocessed_brain_mask_applied.nii.gz",
    reorient="IAL")
# explore_3D_array_with_mask_contour(anat_file.numpy(), mask_file.numpy())
#print(anat_file)
print(anat_file_origin)
print(anat_file)
print(mask_file_aplied)

explore_3D_array_comparison(
    arr_before=anat_file.numpy(),
    arr_after=mask_file_aplied.numpy(),
    cmap="gray",
)
explore_3D_array(anat_file_origin.numpy())

#Procesamiento individual


In [ ]:
#@title **Normalizacion de voxeles**
import nibabel as nib
from nibabel.processing import resample_to_output
import ants

def resample_and_save_image(input_path, output_path):
    try:
        # Cargar la imagen MRI
        img = nib.load(input_path)

        # Crear una nueva imagen con el nuevo espaciado
        new_img = resample_to_output(img, voxel_sizes=(1.0, 1.0, 1.0))

        # Guardar la nueva imagen con el espaciado modificado
        nib.save(new_img, output_path)

        return "La imagen se guardó correctamente."

    except Exception as e:
        return f"Error al procesar la imagen: {str(e)}"

# Ejemplo de uso de la función
ruta_entrada = 'neurocon/data/raw/control/sub-control032014/anat/sub-control032014_T1w.nii.gz'
ruta_salida = 'ruta_a_tu_nueva_imagen.nii.gz'

resultado = resample_and_save_image(ruta_entrada, ruta_salida)
print(resultado)
#Verificar si el espaciado se guardo correctamente
print('Imegen de salida: ',ants.image_read(ruta_salida, reorient="IAL"))
print('Imagen de entreada: ',ants.image_read(ruta_entrada, reorient="IAL"))


In [ ]:
#@title **Obtener mascara para el skullstriping**

import os
import ants
import SimpleITK as sitk

def create_mask(input_image, output_mask):
    try:
        # Leer la imagen de entrada
        input_img = ants.image_read(input_image, reorient="IAL")

        # Realizar extracción de cerebro
        prob_brain_mask = brain_extraction(input_img, modality="t1", verbose=True)
        brain_mask = ants.get_mask(prob_brain_mask, low_thresh=0.5)

        # Guardar la máscara del cerebro
        ants.image_write(brain_mask, output_mask)
        print(f"Máscara del cerebro creada en: {output_mask}")

        return True

    except Exception as e:
        # Capturar cualquier excepción y mostrar un mensaje de error
        print(f"Error al crear la máscara del cerebro: {e}")
        return False

input_image = "ruta_a_tu_nueva_imagen.nii.gz"
output_mask = "ruta_a_tu_nueva_mascara.nii.gz"
#Verficar el resultado
resultado = create_mask(input_image, output_mask)
print(resultado)
explore_3D_array_with_mask_contour(ants.image_read(input_image, reorient="IAL").numpy(), ants.image_read(output_mask, reorient="IAL").numpy())

In [ ]:
#@title **Skullstriping apartir de la mascara**

import os
import ants
import SimpleITK as sitk

def skull_stripping(input_image, mask_image, output_stripped):
    try:
        # Leer la imagen de entrada y la máscara del cerebro
        input_img = ants.image_read(input_image, reorient="IAL")
        brain_mask = ants.image_read(mask_image, reorient="IAL")

        # Aplicar skull stripping
        masked = ants.mask_image(input_img, brain_mask)

        # Guardar la imagen con skull stripping aplicado
        ants.image_write(masked, output_stripped)
        print(f"Skull stripping aplicado y guardado en: {output_stripped}")

        return True

    except Exception as e:
        # Capturar cualquier excepción y mostrar un mensaje de error
        print(f"Error al aplicar skull stripping: {e}")
        return False
#Verificar el resultado
input_image = "ruta_a_tu_nueva_imagen.nii.gz"
mask_image = "ruta_a_tu_nueva_mascara.nii.gz"
output_stripped = "ruta_a_tu_nueva_imagen_stripped.nii.gz"
resultado = skull_stripping(input_image, mask_image, output_stripped)
print(resultado)
explore_3D_array_comparison(ants.image_read(input_image, reorient="IAL").numpy(), ants.image_read(output_stripped, reorient="IAL").numpy(), cmap="gray")

In [ ]:
#@title **Parcelacion hacia un atlas**
import nibabel as nib
import nilearn.image as nimg
from nilearn import plotting as nplot

def perform_parcellation(input_path, atlas_path, output_path):
    try:
        # Cargar la imagen y el atlas
        img = nib.load(input_path)
        atlas = nib.load(atlas_path)

        # Realizar la parcelación (resampling)
        parcelation = nimg.resample_to_img(atlas, img, interpolation='nearest')

        # Guardar la parcelación en el archivo de salida
        nib.save(parcelation, output_path)
        print(f"La parcelación se guardó correctamente en: {output_path}")

        return True

    except Exception as e:
        # Capturar cualquier excepción y mostrar un mensaje de error
        print(f"Error al realizar la parcelación: {e}")
        return False

# Ejemplo de uso de la función
img_path = 'ruta_a_tu_nueva_imagen.nii.gz'
atlas_path = "/media/yeferson/40b51645-c6d9-4b5d-bbfd-d78fda0fec41/BIV2LAB-1/MRIcroGL/Resources/atlas/PD25-subcortical-1mm.nii.gz"
output_path = 'ruta_a_tu_nueva_imagen_parcelation.nii.gz'

resultado = perform_parcellation(img_path, atlas_path, output_path)

img_parcelation = nib.load(output_path)
nplot.plot_roi(img_parcelation, img.slicer[:, :, 54:55])


In [ ]:
import os
import glob

# Directorios de entrada
parkinson_directory = "/home/franklin_pupils/franklin_pupils/yeferson/Parkinson"
control_directory = "/home/franklin_pupils/franklin_pupils/yeferson/Control"

# Prefijos para normalización y parcelación
normalization_prefix = "normalizacion"
parcellation_prefix = "parcelacion"

# Función para procesar archivos en un directorio
def process_files_in_directory(input_directory):
    # Obtener la lista de archivos en el directorio
    files = glob.glob(os.path.join(input_directory, '**', '*.nii.gz'), recursive=True)

    # Procesar cada archivo
    for input_file in files:
        # Directorio de salida
        output_directory = os.path.dirname(input_file)

        # Ruta de salida para normalización
        output_normalization_path = os.path.join(output_directory, f"{normalization_prefix}_{os.path.basename(input_file)}")

        # Llamar a la función de normalización
        resultado_normalizacion = resample_and_save_image(input_file, output_normalization_path)
        print(resultado_normalizacion)

        # Ruta de salida para parcelación
        output_parcellation_path = os.path.join(output_directory, f"{parcellation_prefix}_{os.path.basename(input_file)}")

        # Llamar a la función de parcelación
        resultado_parcelacion = perform_parcellation(output_normalization_path, atlas_path, output_parcellation_path)
        if resultado_parcelacion:
            print(f"La parcelación se guardó correctamente en: {output_parcellation_path}")
        else:
            print("Hubo un error en la parcelación.")

# Procesar archivos en las carpetas Parkinson y Control
process_files_in_directory(parkinson_directory)
process_files_in_directory(control_directory)


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import nibabel as nib

# Directorios de entrada
parkinson_directory = "/home/franklin_pupils/franklin_pupils/yeferson/Parkinson"
control_directory = "/home/franklin_pupils/franklin_pupils/yeferson/Control"

# Prefijo para parcelación y DataFrame
parcellation_prefix = "parcelacion"

# Función para calcular volúmenes y generar DataFrame
def calcular_volúmenes_y_dataframe(parcelation_path, condition):
    try:
        # Cargar la imagen parcelada
        parcelation_img = nib.load(parcelation_path)
        parcelation_data = parcelation_img.get_fdata()

        # Calcular el volumen de cada región
        region_labels, region_counts = np.unique(parcelation_data, return_counts=True)
        region_volumes = dict(zip(region_labels, region_counts))

        # Eliminar el fondo si está etiquetado como 0
        region_volumes.pop(0, None)

        # Crear un DataFrame
        df = pd.DataFrame([region_volumes])

        # Agregar la columna de nombre
        df['nombre'] = os.path.basename(parcelation_path)

        # Agregar la columna de condición (Parkinson o Control)
        df['condicion'] = condition

        return df

    except Exception as e:
        return pd.DataFrame(columns=['nombre', 'condicion'])

# Función para procesar archivos en un directorio y generar DataFrame
def process_files_and_create_dataframe(input_directory, condition):
    # Obtener la lista de archivos en el directorio
    files = glob.glob(os.path.join(input_directory, '**', f'{parcellation_prefix}*.nii.gz'), recursive=True)

    # Crear un DataFrame vacío para almacenar los resultados
    df_result = pd.DataFrame(columns=['nombre', 'condicion'])

    # Procesar cada archivo y agregar al DataFrame
    for parcelation_file in files:
        df_result = pd.concat([df_result, calcular_volúmenes_y_dataframe(parcelation_file, condition)], ignore_index=True)

    return df_result

# Crear DataFrames para las carpetas Parkinson y Control
df_parkinson = process_files_and_create_dataframe(parkinson_directory, 'Parkinson')
df_control = process_files_and_create_dataframe(control_directory, 'Control')

# Combinar los DataFrames en uno solo
df_combined = pd.concat([df_parkinson, df_control], ignore_index=True)

# Mostrar el DataFrame
print(df_combined)
